# Diluted Unitary Fitting

In [5]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from scipy.stats import gaussian_kde
from quantum_circuits import *
from analysis import *
from utils import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [6]:
def optimize_annulus_distance(spectrum_target = None, 
                              d = None, 
                              rank_start = 1,
                              rank_step = 1,
                              num_iter = 1000,
                              lr = 0.001): 
    
    model_list = []
    distance_list = []
    counter = 0
    
    for rank in range(rank_start, d**2+rank_step, rank_step): 
        model = ModelQuantumMap(channel = DilutedKrausMap(U=generate_unitary(d), 
                                                          c=0.5, 
                                                          kraus_part=KrausMap(d, 
                                                                              rank=rank, 
                                                                              trainable=False),
                                                     ),
                            loss_function = AnnulusDistance(),
                            optimizer = tf.optimizers.Adam(learning_rate=lr),
                            logger = Logger(loss_function = AnnulusDistance(), 
                                            verbose=True,),
                            )
        model.train(inputs = None,
                    targets = [spectrum_target],
                    num_iter = num_iter,
                    
                   )

        model_list.append(model)
        distance_list.append(model.logger.loss_train_list[-1])
        
        idx = np.argmin(distance_list)
        if distance_list[-1] > distance_list[idx]:
            counter += 1
        if counter > 4:
            print("Done!")
            break

    distance_best = distance_list[idx]
    model_best = model_list[idx]

    return distance_best, model_best

def fit_diluted_unitary(spectrum_target_list, 
                        d, 
                        rank_start, 
                        rank_step,
                        num_iter = 1000,
                        lr = 0.001):
    
    distance_best_list = []
    model_best_list = []
    for spectrum_target in tqdm(spectrum_target_list):
        U = generate_haar_random(d, rng) 
        distance_best, model_best = optimize_annulus_distance(spectrum_target,
                                                              d,
                                                              rank_start,
                                                              rank_step,
                                                              num_iter = num_iter,
                                                              lr = lr)
        distance_best_list.append(distance_best)
        model_best_list.append(model_best)
    
    for model in model_best_list:
        model.optimizer = None
        
    return distance_best_list, model_best_list

## Three Qubits

In [7]:
[channel_list1, channel_list2, channel_list3, channel_list4, channel_list5, channel_list6, channel_list7] = loader(data_path("channel_fullRank_belem_longTrain.p"))

spectrum_target_list1 = [channel_spectrum(channel, keep_real=False) for channel in channel_list1]
spectrum_target_list2 = [channel_spectrum(channel, keep_real=False) for channel in channel_list2]
spectrum_target_list3 = [channel_spectrum(channel, keep_real=False) for channel in channel_list3]
spectrum_target_list4 = [channel_spectrum(channel, keep_real=False) for channel in channel_list4]
spectrum_target_list5 = [channel_spectrum(channel, keep_real=False) for channel in channel_list5]
spectrum_target_list6 = [channel_spectrum(channel, keep_real=False) for channel in channel_list6]
spectrum_target_list7 = [channel_spectrum(channel, keep_real=False) for channel in channel_list7]

In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list1, 
                                                          d=d, 
                                                          rank_start = 1,
                                                          rank_step = 1,
                                                          num_iter = 2000,
                                                         )

saver([distance_best_list, model_best_list], data_path("diluted_fitting_8layer.model"))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.25870827764787613 None
0.23123761210177746 None
0.19316427390240198 None
0.14635843343264948 None
0.11023732618505148 None
0.08122688512243606 None
0.055627620437122896 None
0.03636882797064006 None
0.03604765534084946 None
0.03602321840519786 None
0.03600890490859433 None
0.03602504852334844 None
0.03602667659961456 None
0.03604553216827028 None
0.03603218197942683 None
0.0360224349717707 None
0.03600504196386836 None
0.03601676125307163 None
0.036016454919517285 None
0.03604879392730845 None
tf.Tensor(0.0360444036353184, shape=(), dtype=float64)
0.036071301919824106 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.2344712635121265 None
0.20457468696272116 None
0.1747582934940749 None
0.14433240908834333 None
0.11483444324923542 None
0.08780658745198973 None
0.06252375962245804 None
0.03851567137121013 None
0.015937625340388475 None
0.013160353833535064 None
0.013151594602666308 None
0.013145094647663426 None
0.013189820215710288 None
0.013166641816282368 None
0.013189813805086083 None
0.013145612248788772 None
0.013180590710874181 None
0.013146110948448069 None
0.013177689754138081 None
0.013144205564876676 None
tf.Tensor(0.013171305696179757, shape=(), dtype=float64)
0.013154785376656222 None


...

0.22433621760799938 None
0.19566884358818004 None
0.16661335350602235 None
0.13980257227107706 None
0.11432507298576329 None
0.0902831301181077 None
0.06662154240028428 None
0.04381589047789701 None
0.022105746116861696 None
0.00840253917011524 None
0.008332576203947244 None
0.008289813679266613 None
0.008286136946360384 None
0.008270896192156799 None
0.00828096264717275 None
0.008258324602920931 None
0.008294515689331842 None
0.008278690533095273 None
0.008261612423586208 None
0.008265848483282311 None
tf.Tensor(0.008271842306833763, shape=(), dtype=float64)
0.008283355942985059 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.21158173761898497 None
0.18409247693752054 None
0.1573640026191523 None
0.13134740158065544 None
0.10608333337209613 None
0.08165004407074859 None
0.05814155080887589 None
0.035643899956412775 None
0.014222300386151588 None
0.006906144292832839 None
0.006897369576053426 None
0.006957530926194026 None
0.006889816463187591 None
0.006935891795709187 None
0.0069339590002409374 None
0.0069165184366274095 None
0.006894674209386812 None
0.006884861953929905 None
0.006926438706605672 None
0.006896176096188944 None
tf.Tensor(0.006903824282314652, shape=(), dtype=float64)
0.006915673645077053 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.2100776889662416 None
0.182980384559011 None
0.15632296824338943 None
0.13017359867098455 None
0.10465096291519324 None
0.0798913702841904 None
0.056114572308641496 None
0.033544260461144865 None


In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

[distance_best_list, model_best_list] = loader(data_path("diluted_fitting_8layer.model"))
rank_start = model_best_list[-1].channel.rank

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list2,
                                                          d=d, 
                                                          rank_start = rank_start,
                                                          rank_step = 1,
                                                          num_iter = 2000,
                                                          )

saver([distance_best_list, model_best_list], data_path("diluted_fitting_16layer.model"))

In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

[distance_best_list, model_best_list] = loader(data_path("diluted_fitting_16layer.model"))
rank_start = model_best_list[-1].channel.rank

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list3, 
                                                          d=d, 
                                                          rank_start = rank_start,
                                                          rank_step = 1,
                                                          num_iter = 2000,
                                                         )

saver([distance_best_list, model_best_list], data_path("diluted_fitting_24layer.model"))

In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

[distance_best_list, model_best_list] = loader(data_path("diluted_fitting_24layer.model"))
rank_start = model_best_list[-1].channel.rank

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list4, 
                                                          d=d, 
                                                          rank_start = rank_start,
                                                          rank_step = 1,
                                                          num_iter = 2000,
                                                         )

saver([distance_best_list, model_best_list], data_path("diluted_fitting_32layer.model"))

## Four Qubits

In [51]:
n = 4
d = 2**n

[model1, model2, model3, model4, model5] = loader(data_path("belem_4qubit_repeated.model"))

spectrum1 = channel_spectrum(model1.channel, keep_real=False)
spectrum2 = channel_spectrum(model2.channel, keep_real=False) 
spectrum3 = channel_spectrum(model3.channel, keep_real=False)
spectrum4 = channel_spectrum(model4.channel, keep_real=False)
spectrum5 = channel_spectrum(model5.channel, keep_real=False)

In [52]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum1, 
                                                      d = d, 
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 2000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_8layer.model"))

  0%|          | 0/2000 [00:00<?, ?it/s]

0.23604491452368892 None
0.22884847262278685 None
0.2308204072016827 None
0.2269173756274781 None
0.230601897437974 None
0.22670672874034004 None
0.2306018318002978 None
0.23444400456919537 None
0.22670672323330907 None
0.22691681561509486 None
0.23444400456180223 None
0.22670672323292876 None
0.22670672323292676 None
0.2344440045617928 None
0.226706723232917 None
0.23060183149402394 None
0.22670672323292737 None
0.22670672323292754 None
0.22670672323292426 None
0.22691681561509133 None
0.22670672323293137 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.13211728892711652 None
0.124228269786473 None
0.11516116005998811 None
0.10933154211595487 None
0.1007590631779306 None
0.1020446677115946 None
0.09806631227291192 None
0.09807667062026788 None
0.10206560353422467 None
0.09809396709413733 None
0.09807322823898597 None
0.09807324582872091 None
0.10201627208935242 None
0.10200940643050527 None
0.0980711189281119 None
0.10207734989017395 None
0.0980772114548718 None
0.10202302114940087 None
0.09806542467331243 None
0.10201251468934347 None
0.09833033470461072 None


  0%|          | 0/2000 [00:00<?, ?it/s]

...

0.08136982256630776 None
0.05857344712459964 None
0.03610063596701392 None
0.018257488329805442 None
0.018104774201700334 None
0.018024514684499345 None
0.018051279782065707 None
0.018030857697683558 None
0.01802563034144089 None
0.01805042354171248 None
0.018092003677805238 None
0.018063295193634872 None
0.01802519955894599 None
0.01806962508541522 None
0.018038132103454337 None
0.01807190321707214 None
0.018031543627802796 None
0.018058205004148592 None
0.018024771438498875 None
0.01806715640780582 None
0.0180318085793152 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.08032378761980136 None
0.05740702325284701 None
0.03487986558765667 None
0.0200874761599087 None
0.016020629310805178 None
0.016017924771481395 None
0.019989096185142934 None
0.016015505436882915 None
0.016046264293791062 None
0.01605662650597331 None
0.019983978637276693 None
0.01603899851758109 None
0.016048482011390804 None
0.01604632862291908 None
0.01602759012197267 None
0.016015628944414786 None
0.01604437911258159 None
0.016019040999524205 None
0.01999694344884983 None
0.01605719841977147 None
0.01603916792602778 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.07982125176348705 None
0.0568780824094206 None
0.03423919057182126 None
0.01644234684843484 None
0.016151511355372093 None
0.016425423526359388 None
0.016140273614819144 None
0.016136990300821497 None
0.020166723176405152 None
0.020123995795816408 None
0.016189687009168226 None
0.01645151238009128 None
0.016230271759369905 None
0.020120361805504507 None
0.01641617302456725 None
0.01617258550168756 None
0.016136289333611588 None
0.016408601240200247 None
0.016150165418067818 None
0.020081662924222615 None
0.02010576459378485 None
Done!


In [57]:
[distance_best, model_best] = loader(data_path("diluted_fitting_4qubit_8layer.model"))
rank_start = model_best.channel.rank

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum2, 
                                                      d = d, 
                                                      rank_start = rank_start, 
                                                      rank_step = 1,
                                                      num_iter = 2000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_16layer.model"))

  0%|          | 0/2000 [00:00<?, ?it/s]

0.22495338925379235 None
0.20592291881023472 None
0.17939011477342898 None
0.1573734219386585 None
0.136064922073722 None
0.11964866619940799 None
0.09654669927268086 None
0.07851703173199197 None
0.06568223500730272 None
0.050140277118375196 None
0.04032028170352006 None
0.03630860085017336 None
0.04021022967868427 None
0.03629672094697175 None
0.036309394301381304 None
0.04026547313117041 None
0.044217205052159786 None
0.040217734037054934 None
0.03629887882969353 None
0.04025293187235309 None
0.04030593114461766 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.2268853171640712 None
0.20399710057174714 None
0.18143608437148642 None
0.15939381984345483 None
0.13806559164728646 None
0.12162355317000981 None
0.1023274906573512 None
0.08040400903853553 None
0.06378773580322676 None
0.048433834281983834 None
0.036930033134527515 None
0.03685794075527968 None
0.036861679702227916 None
0.0368374751409416 None
0.03683403549282919 None
0.0368342027290835 None
0.0368336313852852 None
0.0368540444508474 None
0.0368470138480476 None
0.03684732025736097 None
0.036864013384083426 None


  0%|          | 0/2000 [00:00<?, ?it/s]

...

0.2250746378728457 None
0.2019510863658441 None
0.17904538277994048 None
0.15655053546586162 None
0.13468992740711772 None
0.11364390116333731 None
0.0935212919151603 None
0.07439138120959078 None
0.05643642898768477 None
0.03963913315757133 None
0.03529005314535409 None
0.035226597208684665 None
0.03916132738820293 None
0.031256392886886475 None
0.031256068993637365 None
0.0312659226601944 None
0.03516543744716672 None
0.03517774234181919 None
0.03520476938097892 None
0.031299391687512154 None
0.03130389468504964 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.22417374582383912 None
0.20101237780088638 None
0.17806251206480067 None
0.1555059454962277 None
0.13352685285416185 None
0.11643282405430323 None
0.09235940519199712 None
0.07720320697030217 None
0.059048512178309406 None
0.03811320679542142 None
0.02932919441788309 None
0.029339777969137955 None
0.02933363929237526 None
0.029342375941958203 None
0.029318465857474878 None
0.02934943524413568 None
0.029435120725813547 None
0.029359959267704062 None
0.02932260841640444 None
0.029420515686272596 None
0.02932473275902851 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.2245252529399885 None
0.20134750243887006 None
0.1784143973604095 None
0.1559038978931286 None
0.1339919386934055 None
0.11293646639484693 None
0.09278691340908365 None
0.07760556296054605 None
0.05566411309754934 None
0.03892117255724433 None
0.03441566824196974 None
0.03439530946330886 None
0.03438627956375046 None
0.03046776705828532 None
0.0304599672269053 None
0.030483871215325527 None
0.03046618807195068 None
0.030460533404443067 None
0.0344089240581975 None
0.030465599101836313 None
0.03051174168119342 None
Done!


In [58]:
[distance_best, model_best] = loader(data_path("diluted_fitting_4qubit_16layer.model"))
rank_start = model_best.channel.rank

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum3, 
                                                      d = d, 
                                                      rank_start = rank_start, 
                                                      rank_step = 1,
                                                      num_iter = 2000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_24layer.model"))

  0%|          | 0/2000 [00:00<?, ?it/s]

0.34522338875503794 None
0.32212897969342263 None
0.2992728084816191 None
0.27683985942128303 None
0.25500983394038784 None
0.23400498035396708 None
0.2178835036358735 None
0.19511849730595623 None
0.18126824002677247 None
0.1645409424318744 None
0.1494530129340183 None
0.13539211416465977 None
0.12273039155766909 None
0.11173843972770817 None
0.09790395198932274 None
0.09671884248208724 None
0.0853814741063227 None
0.08254087736273824 None
0.0761964804122462 None
0.06678643042802705 None
0.058328831776440195 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.3440000704865079 None
0.3208596819617661 None
0.2979494187371577 None
0.2754768138065374 None
0.2536331056652528 None
0.23255692840561407 None
0.2160859194721695 None
0.19710592742829672 None
0.17891288832433883 None
0.16214561997830804 None
0.15454863340450906 None
0.13281195959836306 None
0.1240790579695324 None
0.10493239932608624 None
0.09830800304084672 None
0.08521836626760618 None
0.08086051029376377 None
0.07355267196992288 None
0.06369619237405924 None
0.06326627434231716 None
0.06079708754702594 None


  0%|          | 0/2000 [00:00<?, ?it/s]

...

0.345904650608911 None
0.3224821020255033 None
0.29947221015672626 None
0.27685915079816054 None
0.25110338719636427 None
0.22979671376223937 None
0.21308693081468713 None
0.19366347460909356 None
0.17529496972832848 None
0.15433054605455215 None
0.14190792884818623 None
0.12719114450991276 None
0.11328338119617047 None
0.09691449878754138 None
0.08542739848822235 None
0.07608325856067565 None
0.067679116887546 None
0.060042326308024606 None
0.05746950085605528 None
0.04811607412975298 None
0.05206256252697154 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.34225602954444984 None
0.3190753258390167 None
0.29612744411291325 None
0.27360614153361507 None
0.2516899275366446 None
0.23055616231397988 None
0.21026749419176638 None
0.19470525638334765 None
0.1765019090650759 None
0.15570372524637885 None
0.14365388162341805 None
0.1289401902720635 None
0.11163382868583399 None
0.10312732242703086 None
0.0922622176294622 None
0.08610471331435887 None
0.06993071313638034 None
0.06283188291927715 None
0.05653426321715595 None
0.05930465501833346 None
0.05923185705520085 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.34313570764459245 None
0.3238349314944129 None
0.3008624979204487 None
0.2781146776039342 None
0.25244713389336904 None
0.23126342367352418 None
0.2109802218408208 None
0.19164514815053244 None
0.17702189687375894 None
0.16002027916634748 None
0.1400924364100001 None
0.12884210525080675 None
0.11530312026272584 None
0.10264378805523822 None
0.09114856780849989 None
0.08079537625383479 None
0.06794622095269406 None
0.06377403838824497 None
0.05699501975162755 None
0.052484795180429 None
0.05247758106519796 None
Done!


In [59]:
[distance_best, model_best] = loader(data_path("diluted_fitting_4qubit_24layer.model"))
rank_start = model_best.channel.rank

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum4, 
                                                      d = d, 
                                                      rank_start = rank_start, 
                                                      rank_step = 1,
                                                      num_iter = 2000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_32layer.model"))

  0%|          | 0/2000 [00:00<?, ?it/s]

0.40793744070169036 None
0.3848022357788403 None
0.36189851444793036 None
0.3394109718346353 None
0.317460224094706 None
0.29618913822002774 None
0.27582464542645735 None
0.256454887099798 None
0.23809747013756044 None
0.22082017132890866 None
0.2047192756694338 None
0.18989249328612492 None
0.176319246763437 None
0.16394648359844488 None
0.15267130835184864 None
0.14251134076490107 None
0.13348113594368055 None
0.12551886378834604 None
0.11884985179565843 None
0.11329617900946445 None
0.1079855741152636 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.4071467183598084 None
0.383969075037464 None
0.36102255645048376 None
0.33850134048052544 None
0.3204730884203261 None
0.29927781813227217 None
0.2759354790690611 None
0.2557325079951979 None
0.23834126634231212 None
0.22035188307682052 None
0.20446498572532665 None
0.19366178790620076 None
0.18021409458181883 None
0.1641642013291556 None
0.15316054007677224 None
0.1509322417776439 None
0.1383645107314923 None
0.13430676387314144 None
0.11990423966943788 None
0.1145988027175991 None
0.10835439237620467 None


  0%|          | 0/2000 [00:00<?, ?it/s]

...

0.407530552827276 None
0.3842829261522966 None
0.3612385217928822 None
0.3385778297262767 None
0.3164749713342922 None
0.2950915225126453 None
0.2745734325577033 None
0.25500464654080385 None
0.23648216222403973 None
0.21906664224312872 None
0.20287622352004087 None
0.18854523169749482 None
0.17352243884244067 None
0.16144214935043597 None
0.15261200718748208 None
0.1379081239002104 None
0.1291063170464028 None
0.11957712620734566 None
0.1115835594127759 None
0.10854281616250057 None
0.09855061279202343 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.4080103052115893 None
0.38476380984787295 None
0.36171793745220077 None
0.3429429747196944 None
0.3208350944943322 None
0.29557459529743507 None
0.2750809100864695 None
0.2555264206071161 None
0.2369490449868942 None
0.21941390843113692 None
0.20300577864648212 None
0.1886058622496182 None
0.1734445327417877 None
0.16105685330616717 None
0.15186582370627286 None
0.13776324252589153 None
0.13078992428568015 None
0.11790899725582774 None
0.11075446279062257 None
0.10260709881617956 None
0.09722319274773336 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.41075954561056244 None
0.38662640305965923 None
0.3635619567210206 None
0.3408720765059143 None
0.3196117576305696 None
0.29732175253201787 None
0.2767255932537936 None
0.25701821345199366 None
0.23829703370293479 None
0.22148241388750983 None
0.20487526653145138 None
0.19242254044681129 None
0.1740822715300335 None
0.16071271909646725 None
0.14856851736619364 None
0.13753675002955518 None
0.13152939252661128 None
0.11964983204188835 None
0.11482091219985793 None
0.10777730700855911 None
0.09770007265953028 None
Done!
